In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
#default_exp examples.dummy_experiment_manager
from nbdev.showdoc import show_doc
import warnings
warnings.filterwarnings('ignore')

# Dummy Experiment Manager

> Dummy experiment manager

In [ ]:
#export
import numpy as np
import pickle

class FakeModel (object):
    
    overfitting_epochs = 20
    
    def __init__ (self, offset=0.5, rate=0.01, epochs=10, noise=0.0, verbose=True):
        # hyper-parameters
        self.offset = offset
        self.rate = rate
        self.epochs = epochs
        
        # fake internal weight
        self.weight = 0
        
        # fake accuracy
        self.accuracy = self.offset
        
        # noise
        self.noise = noise
        
        # other parameters
        self.verbose = verbose
        
        self.history = {}
    
    def fit (self):
        number_epochs = int(self.epochs)
        if self.verbose:
            print (f'fitting model with {number_epochs} epochs')
        for epoch in range(number_epochs):
            self.weight += self.rate
            if epoch < self.overfitting_epochs:
                self.accuracy += self.rate
            else:
                self.accuracy -= self.rate
            if self.verbose:
                print (f'epoch {epoch}: accuracy: {self.accuracy}')
            
            # we keep track of the evolution of different metrics to later be able to visualize it
            self.store_intermediate_metrics ()
    
    def store_intermediate_metrics (self):
        validation_accuracy, test_accuracy = self.score()
        if 'validation_accuracy' not in self.history:
            self.history['validation_accuracy'] = []
        self.history['validation_accuracy'].append(validation_accuracy)
        
        if 'test_accuracy' not in self.history:
            self.history['test_accuracy'] = []
        self.history['test_accuracy'].append(test_accuracy)
        
        if 'accuracy' not in self.history:
            self.history['accuracy'] = []
        self.history['accuracy'].append(self.accuracy)
        
    def save_model_and_history (self, path_results):
        pickle.dump (self.weight, open(f'{path_results}/model_weights.pk','wb'))
        pickle.dump (self.history, open(f'{path_results}/model_history.pk','wb'))
        
    def load_model_and_history (self, path_results):
        self.weight = pickle.load (open(f'{path_results}/model_weights.pk','rb'))
        self.history = pickle.load (open(f'{path_results}/model_history.pk','rb'))
        
    def score (self):
        # validation accuracy
        validation_accuracy = self.accuracy + np.random.randn() * self.noise
        
        # test accuracy
        if self.epochs < 10:
            test_accuracy = self.accuracy + 0.1
        else:
            test_accuracy = self.accuracy - 0.1
        test_accuracy = test_accuracy + np.random.randn() * self.noise
        
        # make accuracy be in interval [0,1] 
        validation_accuracy = max(min(validation_accuracy, 1.0), 0.0)
        test_accuracy = max(min(test_accuracy, 1.0), 0.0)
        
        return validation_accuracy, test_accuracy
    
    # fake load_data which does nothing
    def load_data (self):
        pass

In [ ]:
#export
from hpsearch.experiment_manager import ExperimentManager
import hpsearch
import os

class DummyExperimentManager (ExperimentManager):

    def __init__ (self):
        super().__init__()

    def run_experiment (self, parameters={}, path_results='./results'):
        # extract hyper-parameters used by our model. All the parameters have default values if they are not passed.
        offset = parameters.get('offset', 0.5)   # default value: 0.5
        rate = parameters.get('rate', 0.01)   # default value: 0.01
        epochs = parameters.get('epochs', 10) # default value: 10
        noise = parameters.get('noise', 0.0)
        
        # other parameters that do not form part of our experiment definition
        # changing the values of these other parameters, does not make the ID of the experiment change
        verbose = parameters.get('verbose', True)
        
        # build model with given hyper-parameters
        model = FakeModel (offset=offset, rate=rate, epochs=epochs, noise = noise, verbose=verbose)
        
        # load training, validation and test data (fake step)
        model.load_data()
        
        # fit model with training data 
        model.fit ()
        
        # save model weights and evolution of accuracy metric across epochs
        model.save_model_and_history(path_results)
        
        # evaluate model with validation and test data
        validation_accuracy, test_accuracy = model.score()
        
        # the function returns a dictionary with keys corresponding to the names of each metric. 
        # We return result on validation and test set in this example
        dict_results = dict (validation_accuracy = validation_accuracy,
                             test_accuracy = test_accuracy)
        
        return dict_results
    
    # implementing the following method is not necessary but recommended
    def get_default_parameters (self, parameters):
        """Indicate the default value for each of the hyper-parameters used."""
        defaults = dict(offset=0.5,
                        rate=0.01,
                        epochs=10)
        return defaults
    
    # implementing the following method is not necessary but recommended
    def get_path_experiments (self, path_experiments = None, folder = None):
        """Gives the root path to the folder where results of experiments are stored."""
        path_experiments = f'{os.path.dirname(hpsearch.__file__)}/../results'
        if folder != None:
            path_experiments = f'{path_experiments}/{folder}'
        return path_experiments
    
    # implementing the following method is not necessary but recommended
    def get_default_operations (self):
        default_operations = dict (root='',
                                   metric='validation_accuracy',
                                   op='max')
        
        return default_operations
    

## Example use

In [ ]:
# export
def run_multiple_experiments (nruns=1, noise=0.0, verbose=True, rate=0.03):
    em = DummyExperimentManager ()
    parameters_single_value = dict(rate=rate, noise=noise)   # parameters where we use a fixed value
    parameters_multiple_values=dict(offset=[0.1, 0.3, 0.6], epochs=[5, 15, 30]) # parameters where we try multiple values
    other_parameters = dict(verbose=verbose) # parameters that control other aspects that are not part of our experiment definition (a new experiment is not created if we assign different values for these parametsers)
    em.grid_search (log_message='fixed rate, multiple epochs values',
            parameters_single_value=parameters_single_value,
            parameters_multiple_values=parameters_multiple_values,
            other_parameters=other_parameters,
            nruns=nruns)


In [ ]:
# export 
import shutil
import os

def remove_previous_experiments():
    em = DummyExperimentManager ()
    path_results = em.get_path_experiments()
    if os.path.exists(path_results):
        shutil.rmtree(path_results)

In [ ]:
remove_previous_experiments()
run_multiple_experiments()

experiment script: <ipython-input-78-3199ad639018>, line: 11
processing hyper-parameter 0 out of 9
doing run 0 out of 1
fixed rate, multiple epochs values
running experiment 0
run number: 0

parameters:
	epochs=5,
	noise=0.0,
	offset=0.1,
	rate=0.03

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.0009465217590332031
0 - validation_accuracy: 0.25
0 - test_accuracy: 0.35
finished experiment 0
processing hyper-parameter 1 out of 9
doing run 0 out of 1
fixed rate, multiple epochs values
running experiment 1
run number: 0

parameters:
	epochs=5,
	noise=0.0,
	offset=0.3,
	rate=0.03

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.0009696483612060547
0 - validation_accuracy: 0.45000000000000007
0 - test_accuracy: 0.55
finished experiment 1
processing hyper-parameter 2 out of 9
doing run 0 out of 1
fixed rate, multiple epochs values
running experiment 2
run number: 0

parameters:
	epochs=5,
	noise=0.0,
	offset

current path: /mnt/athena/hpsearch/nbs/examples
current path: /mnt/athena/hpsearch/nbs/examples
write_manager failed with exception <class '__main__.DummyExperimentManager'> is a built-in class
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
write_manager failed with exception <class '__main__.DummyExperimentManager'> is a built-in class
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
write_manager failed with exception <class '__main__.DummyExperimentManager'> is a built-in class
fitting model with 5 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
write_manager failed with exception <class '__main__.DummyExperimentManager'> i

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.002310514450073242
0 - validation_accuracy: 0.7500000000000003
0 - test_accuracy: 0.6500000000000004
finished experiment 4
processing hyper-parameter 5 out of 9
doing run 0 out of 1
fixed rate, multiple epochs values
running experiment 5
run number: 0

parameters:
	epochs=15,
	noise=0.0,
	offset=0.6,
	rate=0.03

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.0019295215606689453
0 - validation_accuracy: 1.0
0 - test_accuracy: 0.9500000000000003
finished experiment 5
processing hyper-parameter 6 out of 9
doing run 0 out of 1
fixed rate, multiple epochs values
running experiment 6
run number: 0

parameters:
	epochs=30,
	noise=0.0,
	offset=0.1,
	rate=0.03

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.003453493118286133
0 - validation_accuracy: 0.40000000000000013
0 - test_accuracy: 0.30000000000000016
finished expe

fitting model with 15 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
write_manager failed with exception <class '__main__.DummyExperimentManager'> is a built-in class
fitting model with 15 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy:

running experiment 8
run number: 0

parameters:
	epochs=30,
	noise=0.0,
	offset=0.6,
	rate=0.03

script: <ipython-input-78-3199ad639018>, line number: 11
time spent on this experiment: 0.003430604934692383
0 - validation_accuracy: 0.9000000000000001
0 - test_accuracy: 0.8000000000000002
finished experiment 8


fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

In [ ]:
import pandas as pd

em = DummyExperimentManager ()
path_results = em.get_path_experiments()
df = pd.read_csv (f'{path_results}/experiments_data.csv', index_col=0)
df

current path: /mnt/athena/hpsearch/nbs/examples


,epochs,noise,offset,rate,0_validation_accuracy,0_test_accuracy,time_0,date,0_finished
0,5.0,0.0,0.1,0.03,0.25,0.35,0.001476,11:11:44.886454,True
1,5.0,0.0,0.3,0.03,0.45,0.55,0.001503,11:11:44.926644,True
2,5.0,0.0,0.6,0.03,0.75,0.85,0.001437,11:11:44.973356,True
3,15.0,0.0,0.1,0.03,0.55,0.45,0.002577,11:11:45.021528,True
4,15.0,0.0,0.3,0.03,0.75,0.65,0.002898,11:11:45.070110,True
5,15.0,0.0,0.6,0.03,1.00,0.95,0.002454,11:11:45.122967,True
6,30.0,0.0,0.1,0.03,0.40,0.30,0.003990,11:11:45.179634,True
7,30.0,0.0,0.3,0.03,0.60,0.50,0.004106,11:11:45.237987,True
8,30.0,0.0,0.6,0.03,0.90,0.80,0.004007,11:11:45.297369,True


In [ ]:
import numpy as np

# check that stored parameters are correct
assert (df.epochs.values == np.array([ 5.,  5.,  5., 15., 15., 15., 30., 30., 30.])).all()
assert (df.offset.values == np.array([0.1, 0.3, 0.6, 0.1, 0.3, 0.6, 0.1, 0.3, 0.6])).all()
assert (df.rate.values == 0.03).all()

In [ ]:
# check that the accuracy values are correct
epochs_before_overfitting = 20
epochs_test = 10
for experiment_id in df.index:
    if df.loc[experiment_id, 'epochs'] < epochs_before_overfitting:
        accuracy = df.loc[experiment_id, 'offset'] + df.loc[experiment_id, 'rate'] * df.loc[experiment_id, 'epochs']
    else:
        epochs_after_overfitting = df.loc[experiment_id, 'epochs']-epochs_before_overfitting
        accuracy = df.loc[experiment_id, 'offset'] + df.loc[experiment_id, 'rate'] * (epochs_before_overfitting  - epochs_after_overfitting)
    if df.loc[experiment_id, 'epochs'] < epochs_test:
        test_accuracy = accuracy + 0.1
    else:
        test_accuracy = accuracy - 0.1
    validation_accuracy = max(min(accuracy, 1.0), 0.0)
    test_accuracy = max(min(test_accuracy, 1.0), 0.0)
    
    assert np.abs(df.loc[experiment_id, '0_validation_accuracy'] - validation_accuracy) <1.e-10, f"experiment {experiment_id}: {df.loc[experiment_id, '0_validation_accuracy']} == {validation_accuracy}" 
    assert np.abs(df.loc[experiment_id, '0_test_accuracy'] - test_accuracy) <1.e-10

check that model history is written correcly

In [ ]:
path_experiment = em.get_path_results (3, 0)
model = FakeModel()
model.load_model_and_history(path_experiment)
assert np.max(np.abs(model.history['accuracy']-np.arange(0.13, 0.55, 0.03))) < 1e-10

In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)

Converted hpconfig.ipynb.
Converted manager_factory.ipynb.
Converted dummy_experiment_manager.ipynb.
Converted example_experiment.ipynb.
Converted example_experiment_manager.ipynb.
Converted experiment_manager.ipynb.
Converted index.ipynb.
Converted change_manager.ipynb.
Converted metric_visualization.ipynb.
Converted print_parameters.ipynb.
Converted query.ipynb.
Converted experiment_utils.ipynb.
Converted organize_experiments.ipynb.
Converted experiment_visualization.ipynb.
Converted plot_utils.ipynb.
